# Imports

In [1]:
import math, pyperclip, os, re
from decimal import Decimal as _Decimal
import numpy as np

# Docs

In [2]:
"""
Data types: Real, Complex, Int, Boolean, String, Undefined
Data structures: Scalar, Vector, Matrix, ...
Variables: v
Functions: fun
Named operators: v operation u
Conversion specifier: 45deg
Data properties: object.property
Oppertation: Numeric, Sets, Comparison 

Named operators
on two values: v operation u
on one value: v.operation

Input and Output conversion
hex : 0b1100 * 3_210 + 15_12E3
datetime : 2h + 45m + 1h + 30m
base 4 : 10

Basic Operations:
Standard:       | Bitwise: int&bool | Comparison:           | 
+           add | ~             not | ==             equals | 
-      subtract | &&            and | !=          not equal | 
*      multiply | ||             or | <           less than | 
/        divide | <xor>         xor | >        greater then | 
//      int div | <<     left shift | <=      less or equal | 
%       modulus | >>    right shift | >=   greater or equal | 
^         power |                   |                       | 
!     factorial |                   |                       | 
|val|  absolute |                   |                       | 
=    assignment |                   |                       | 

Higher Ranking Data Structure Operations:
Vector:                 | Matrix:                          | Reductions:
<dot>       dot product | <matmul>   matrix multiplication | <all>
<cross>   cross product | |mat|                            | <any>
|vec|            length | .T              transpose matrix | <
.length          length | 
.angle            angle | 



1 + sqrt 4
1 + sqrt x
1 + $4
1 + 3 root 8
1 + b root x
1 + b $ x

10 nPr 2
n nPr r
10 nCr 2
n nCr r

[1, 2,, 3, 4] matmul [5, 6,, 7, 8]
[1, 2,, 3, 4] MatMul [5, 6,, 7, 8]
[1, 2,, 3, 4] MATMUL [5, 6,, 7, 8]
[1, 2,, 3, 4] # [5, 6,, 7, 8]
mat1 matmul mat2
mat1 MatMul mat2
mat1 MATMUL mat2
[1, 2,, 3, 4].T
mat1.T

5*x

sin pi
sin2 pi
sini 0.5

@display = scientific 8
@display = hex


Statistical Operations:
"""
None

# Classes

In [3]:
class DType():
    
    def __init__(self, name):
        self.dtype = name
        self.value = None
    
    @staticmethod
    def name(self):
        pass
    
    @staticmethod
    def __str__(self):
        return str(self.va)
    

## Token Matching

### Operands

In [4]:
class Operand():
    def __init__(self, match, precedence):
        self.match = match
        self.re_match = re.escape(match)
        self.precedence = float(precedence)
        
    def __repr__(self):
        return f'Operand({self.re_match}, {self.precedence})'
        
    def __str__(self):
        return f'Operand({self.re_match}, {self.precedence})'

### Groups

In [5]:
class Group():
    def __init__(self, re_match, close_op):
        self.re_match = re_match
        self.close_op = close_op
        
    def __repr__(self):
        return f'Group({self.re_match}, {self.close_op})'
        
    def __str__(self):
        return f'Group({self.re_match}, {self.close_op})'

## Token Tree Nodes

### Abstract NodeToken

In [6]:
class NodeToken():
    """
    represents a token that can be parsed
    """
        
    def dict(self):
        raise Exception('dict() not implemented')
    
    def is_complete(self):
        raise Exception('is_complete() not implemented')
        
    def set_left(self, node):
        raise Exception('set_left(node) not implemented')

    def set_right(self, node):
        raise Exception('set_right(node) not implemented')
        
    def get_right(self):
        raise Exception('get_right() not implemented')

### NodeBinary

In [7]:
class NodeBinary(NodeToken):
    """
    represents a token that can be parsed into a value/operation
    """
    def __init__(self, value, precedence, parent=None):
        self.parent = parent
        self.value = value
        self.precedence = precedence
        self.left = None
        self.right = None
        
    def dict(self):
        return {self.value: {'left': self.left.dict(), 'right': self.right.dict()}}
    
    def is_complete(self):
        return self.left != None and self.right != None
        
    def set_left(self, node):
        self.left = node
        node.parent = self

    def set_right(self, node):
        self.right = node
        node.parent = self
        
    def get_right(self):
        return self.right

### NodeUnaryLeft

In [8]:
class NodeUnaryLeft(NodeToken):
    
    def __init__(self, value, precedence, parent=None):
        self.parent = parent
        self.value = value
        self.precedence = precedence
        self.child = None
        
    def dict(self):
        return {self.value: {'child': self.child.dict()}}
    
    def is_complete(self):
        return self.child != None
        
    def set_left(self, node):
        self.child = node
        node.parent = self

    def set_right(self, node):
        self.child = node
        node.parent = self
        
    def get_right(self):
        return self.child

### NodeUnaryRight

In [9]:
class NodeUnaryRight(NodeToken):
    
    def __init__(self, value, precedence, parent=None):
        self.parent = parent
        self.value = value
        self.precedence = precedence
        self.child = None
        
    def dict(self):
        return {self.value: {'child': self.child.dict()}}
    
    def is_complete(self):
        return self.child != None
        
    def set_left(self, node):
        self.child = node
        node.parent = self

    def set_right(self, node):
        self.child = node
        node.parent = self
        
    def get_right(self):
        return self.child

### NodeGroup

In [10]:
class NodeGroup(NodeToken):
    
    def __init__(self, value, closing_token, parent=None):
        self.parent = parent
        self.value = value
        self.closing_token = closing_token
        self.complete = False
        self.children = []
        
        self.shape = {0:1}
        self.sep_count = 0
        
    def dict(self):
        return {(self.value, self.get_shape()): [c.dict() for c in self.children]}
    
    def is_complete(self):
        return self.complete

    def close(self):
        self.sep_count = 0
        self.complete = True
        
        expected_size = 1
        for s in self.get_shape(): expected_size *= s
            
        if len(self.children) < expected_size:
            raise Exception('Inconsistent dimensions')
    
    def set_right(self, node):
        self.children[-1] = node
        node.parent = self
        
    def get_right(self):
        return self.children[-1]

    def add(self, node):
        self.children.append(node)
        node.parent = self
        
        if self.sep_count >= len(self.shape):
            for i in range(len(self.shape), self.sep_count+1):
                self.shape[i-1] = self.shape.get(i-1, 1)
            self.shape[self.sep_count-1] += 1
            
        self.sep_count = 0
        
    def increase(self):
        self.sep_count += 1
        
        # ensure minumum number of dimensions exists
        for i in range(len(self.shape), self.sep_count+1):
            self.shape[i-1] = self.shape.get(i-1, 1)
        
#         shape_size = 1
#         for d,s in self.shape.items(): shape_size *= s
        
#         dim = shape_size - len(self.children)
#         print(f'shape_size:{shape_size}, children:{len(self.children)}, dim:{dim}')
        
#         if dim+1 >= len(self.shape):
#             if dim > 0:
#                 self.shape[dim-1] = self.shape.get(dim-1, 1) - 1
#             self.shape[dim] = self.shape.get(dim, 1) + 1
#             print(self.shape)
    
    def get_shape(self):
        shape = [(d,s) for d,s in self.shape.items()]
        shape.sort(key=lambda x:x[0], reverse=True)
        shape = tuple(s for d,s in shape)
        return shape
    
    def get_rank(self):
        return len(self.shape)

### NodeValue

In [11]:
class NodeValue(NodeToken):
    
    def __init__(self, value, kind, parent=None):
        self.parent = parent
        self.value = value
        self.kind = kind
        
    def dict(self):
        return self.value
    
    def is_complete(self):
        return True

## Evaluatable Tree Nodes

### Display

### Abstract NodeEvaluable

In [12]:
class NodeEvaluable():
    """
    represents a node that can be evaluated
    """
        
    def dict(self):
        raise Exception('dict() not implemented')
    
    def eval(self, environment):
        raise Exception('eval(environment) not implemented')

In [13]:
class Display():
    def __init__(self, node):
        self.node = node
        
    

### Tensor

In [14]:
# numpy dtypes
# {
# 'int':     [numpy.int8,      numpy.int16,      numpy.int32,  numpy.int64], 
# 'uint':    [numpy.uint8,     numpy.uint16,     numpy.uint32, numpy.uint64], 
# 'float':   [numpy.float16,   numpy.float32,    numpy.float64], 
# 'complex': [numpy.complex64, numpy.complex128], 
# 'others':  [bool, object, bytes, str, numpy.void]
# }

dtypes = {
    'int': np.uint64,
    'real': object, # Decimal
    'complex': np.complex128,
    'bool': bool,
    'str': str,
}

class Tensor():
    """
    represents a value of any type and rank
    """
    def __init__(self, dtype, rank=0, shape=()):
        self.dtype = dtype
        self.rank = rank
        self.shape = shape
        self.value = None
        
    def __get__(self, index):
        return self.value[index]
        
    def __set__(self, index, value):
        self.value[index] = value

# Define Token

## Tokens Types

In [15]:
    
# operand kinds
# group, close_group, return_to_group, binary, unary_left, unary_right, *_ = enum(10)
# (
#     OPERAND_TYPE_GROUP, 
#     OPERAND_TYPE_CLOSE_GROUP, 
#     OPERAND_TYPE_RETURN_TO_GROUP, 
#     OPERAND_TYPE_BINARY, 
#     OPERAND_TYPE_UNARY_LEFT, 
#     OPERAND_TYPE_UNARY_RIGHT, 
# *_) = enum(10)


TOKEN_TYPE_STRING =   'string'
TOKEN_TYPE_INTEGER =  'integer'
TOKEN_TYPE_NUMBER =   'number'
TOKEN_TYPE_OPERATOR = 'operand'
TOKEN_TYPE_GROUP =    'group'
TOKEN_TYPE_LITERAL =  'literal'

TOKEN_TYPE_CLOSE_GROUP = 'close group'
TOKEN_TYPE_NEW_ITEM    = 'new item'
TOKEN_TYPE_BINARY      = 'binary operand'
TOKEN_TYPE_UNARY_LEFT  = 'left unary operand'
TOKEN_TYPE_UNARY_RIGHT = 'right unary operand'

OPERAND_TYPES = [
    TOKEN_TYPE_CLOSE_GROUP, 
    TOKEN_TYPE_NEW_ITEM, 
    TOKEN_TYPE_BINARY, 
    TOKEN_TYPE_UNARY_LEFT, 
    TOKEN_TYPE_UNARY_RIGHT, 
]

TOKEN_TYPE_VALUE = [
    TOKEN_TYPE_STRING,
    TOKEN_TYPE_INTEGER,
    TOKEN_TYPE_NUMBER,
    TOKEN_TYPE_LITERAL,
]

## Token Operations

In [16]:
binary_operands = [
    Operand(r'.',   8), 
    Operand(r':',   6), 
    Operand(r'^',   5), 
    Operand(r'*',   4), 
    Operand(r'/',   4), 
    Operand(r'//',  4), 
    Operand(r'%',   4), 
    Operand(r'+',   3), 
    Operand(r'-',   3), 
    Operand(r'&&',  2), 
    Operand(r'||',  2), 
    Operand(r'xor', 2), 
    Operand(r'==',  1), 
    Operand(r'!=',  1), 
    Operand(r'>',   1), 
    Operand(r'>=',  1), 
    Operand(r'<',   1), 
    Operand(r'<=',  1), 
    Operand(r'=',   0),
]

left_unary_operands = [
    Operand(r'+', 7), 
    Operand(r'-', 7), 
    Operand(r'~', 7), 
    Operand(r'=', 0),
]

right_unary_operands = [
    Operand(r'!', 7),
]

new_item = [
    Operand(r',', -1),
]

groups = [
    Group(r'\(', r')'),
    Group(r'\[', r']'),
    Group(r'\{', r'}'),
    Group(r'[a-zA-Z][a-zA-Z0-9_]*\(', r')'),
]

close_group = [Operand(g.close_op, -1) for g in groups]

# sort by length
binary_operands.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)
left_unary_operands.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)
right_unary_operands.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)
new_item.sort(key = lambda o:(len(o.match), o.precedence), reverse=True)

(
    binary_operands,
    left_unary_operands,
    right_unary_operands,
    new_item,
    close_group,
)

([Operand(xor, 2.0),
  Operand(\/\/, 4.0),
  Operand(\&\&, 2.0),
  Operand(\|\|, 2.0),
  Operand(\=\=, 1.0),
  Operand(\!\=, 1.0),
  Operand(\>\=, 1.0),
  Operand(\<\=, 1.0),
  Operand(\., 8.0),
  Operand(\:, 6.0),
  Operand(\^, 5.0),
  Operand(\*, 4.0),
  Operand(\/, 4.0),
  Operand(\%, 4.0),
  Operand(\+, 3.0),
  Operand(\-, 3.0),
  Operand(\>, 1.0),
  Operand(\<, 1.0),
  Operand(\=, 0.0)],
 [Operand(\+, 7.0), Operand(\-, 7.0), Operand(\~, 7.0), Operand(\=, 0.0)],
 [Operand(\!, 7.0)],
 [Operand(\,, -1.0)],
 [Operand(\), -1.0), Operand(\], -1.0), Operand(\}, -1.0), Operand(\), -1.0)])

## Operation Indexing

In [17]:
# def filter_attribute(items, exact={}, regex={}, reverse_regex={}):
#     filtered = []
    
#     for item in items:
#         match = True
#         for p in exact:
#             match &= getattr(item, p) == exact[p]
#         for p in regex:
#             match &= re.fullmatch(regex[p], getattr(item, p)) != None
#         for p in reverse_regex:
#             match &= re.fullmatch(getattr(item, p), reverse_regex[p]) != None
            
#         if match:
#             filtered.append(item)
            
#     return filtered
    
    
# def find_operand(string, op_kind):
#     filtered = filter_attribute(operands, {'op_kind':op_kind},{},{'re_match':string})
    
#     if len(filtered) == 0:
#         return None
# #         raise Exception(f"Operand '{string}' not found for this context.")
    
#     return filtered[0]
            
        
# def find_group(string):
#     filtered = filter_attribute(groups, {},{},{'re_match':string})
    
#     if len(filtered) == 0:
#         raise Exception(f"Group token '{string}' not found.")
    
#     return filtered[0]


# def get_possible_operation_types(string):
# #     types = [find_operand(string, t) for t in order]
# #     types = [t for t in types if t != None]
#     types = {t:find_operand(string, t) for t in OPERAND_TYPES}
#     types = {t:o.op_kind for t,o in types.items() if o != None}
#     return types

def find_operand(string, operators):
    for operator in operators:
        if re.fullmatch(operator.re_match, string) != None:
            return operator
    return None

## Define Token Regular Expressions

In [18]:
def re_join(l, f=lambda x:x):
    return '(' + ')|('.join([f(i) for i in l]) + ')'

# regex
re_number =    r"""((([\.][0-9_]+)|([0-9_]+[\.]?[0-9_]*))([eE][-+]?[0-9_]+)?[a-z]*)"""
re_integer =   r"""((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)"""
re_string =    r"""((\""".*?\""")|('''.*?''')|(".*?")|('.*?'))"""
re_literal =   r"""([A-Za-z_][A-Za-z0-9_]*)"""

re_groups =               re_join(groups, lambda x:x.re_match)
re_binary_operands =      re_join(binary_operands, lambda x:x.re_match)
re_left_unary_operands =  re_join(left_unary_operands, lambda x:x.re_match)
re_right_unary_operands = re_join(right_unary_operands, lambda x:x.re_match)
re_new_item =             re_join(new_item, lambda x:x.re_match)
re_close_group =          re_join(close_group, lambda x:x.re_match)


re_tokens = {
    TOKEN_TYPE_STRING:      re_string,
    TOKEN_TYPE_INTEGER:     re_integer,
    TOKEN_TYPE_NUMBER:      re_number,
    TOKEN_TYPE_LITERAL:     re_literal,
    TOKEN_TYPE_GROUP:       re_groups,
    TOKEN_TYPE_BINARY:      re_binary_operands,
    TOKEN_TYPE_UNARY_LEFT:  re_left_unary_operands,
    TOKEN_TYPE_UNARY_RIGHT: re_right_unary_operands,
    TOKEN_TYPE_NEW_ITEM:    re_new_item,
    TOKEN_TYPE_CLOSE_GROUP: re_close_group,
}

re_tokens

{'string': '((\\""".*?\\""")|(\'\'\'.*?\'\'\')|(".*?")|(\'.*?\'))',
 'integer': '((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)',
 'number': '((([\\.][0-9_]+)|([0-9_]+[\\.]?[0-9_]*))([eE][-+]?[0-9_]+)?[a-z]*)',
 'literal': '([A-Za-z_][A-Za-z0-9_]*)',
 'group': '(\\()|(\\[)|(\\{)|([a-zA-Z][a-zA-Z0-9_]*\\()',
 'binary operand': '(xor)|(\\/\\/)|(\\&\\&)|(\\|\\|)|(\\=\\=)|(\\!\\=)|(\\>\\=)|(\\<\\=)|(\\.)|(\\:)|(\\^)|(\\*)|(\\/)|(\\%)|(\\+)|(\\-)|(\\>)|(\\<)|(\\=)',
 'left unary operand': '(\\+)|(\\-)|(\\~)|(\\=)',
 'right unary operand': '(\\!)',
 'new item': '(\\,)',
 'close group': '(\\))|(\\])|(\\})|(\\))'}

# Parse

## Lexing

In [19]:
def re_match_length(string, re_pattern):
    match = re.match(re_pattern, string)
    return match.span()[1] if match != None else 0

# TOKEN_TYPE_STRING
# TOKEN_TYPE_INTEGER
# TOKEN_TYPE_NUMBER
# TOKEN_TYPE_LITERAL

# TOKEN_TYPE_GROUP
# TOKEN_TYPE_BINARY
# TOKEN_TYPE_UNARY_LEFT
# TOKEN_TYPE_UNARY_RIGHT
# TOKEN_TYPE_NEW_ITEM
# TOKEN_TYPE_CLOSE_GROUP

TOKEN_TYPE_VALUE = [
    TOKEN_TYPE_STRING,
    TOKEN_TYPE_INTEGER,
    TOKEN_TYPE_NUMBER,
    TOKEN_TYPE_LITERAL,
]

def lexing(string):
    tokens = []

    i = 0
    while i < len(string):
            
        last_token_type = tokens[-1][0] if len(tokens) > 0 else None
        allowed_token_types = []

        # begin with
        if last_token_type in [
            None,
            TOKEN_TYPE_GROUP,
            TOKEN_TYPE_NEW_ITEM,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_LEFT,
                TOKEN_TYPE_GROUP,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
                *TOKEN_TYPE_VALUE,
            ]

        # after value
        if last_token_type in [
            TOKEN_TYPE_CLOSE_GROUP,
            TOKEN_TYPE_UNARY_RIGHT,
            *TOKEN_TYPE_VALUE,
        ]:
            allowed_token_types = [
                TOKEN_TYPE_UNARY_RIGHT,
                TOKEN_TYPE_BINARY,
                TOKEN_TYPE_NEW_ITEM,
                TOKEN_TYPE_CLOSE_GROUP,
            ]

        # after operator
        if last_token_type in [
            TOKEN_TYPE_BINARY,
            TOKEN_TYPE_UNARY_LEFT,

        ]:
            allowed_token_types = [
                TOKEN_TYPE_GROUP,
                TOKEN_TYPE_UNARY_LEFT,
                *TOKEN_TYPE_VALUE,
            ]
            
        
        # find matching token type
        token_type, token_str = None, None
        
        for possible_token_type in allowed_token_types:
            re_pattern = re_tokens[possible_token_type]
            
            l = re_match_length(string[i:], re_pattern)
            if l > 0:
                token_type = possible_token_type
                token_str = string[i:i+l]
                break
                

                    
        # invalid token
        if token_type == None:
#             raise Exception(f"Token not allowed at position {i}")
            i += 1
        else:
            tokens.append((token_type, token_str))
            i += len(token_str)

    return tokens


## Treeify

In [20]:
def bubble_up(focus, new):
    """
    Finds ancestor/parent in tree upwards from `token` thats the first group token or the first
    """
    
    while True:
#         print('bubble_up', focus.value, type(focus))
#         if type(focus) != NodeValue and (type(focus) == NodeGroup or focus.precedence < new.precedence):
        if type(focus) == NodeGroup or focus.precedence < new.precedence:
#             print('return', focus.value)
            return focus
        else:
            focus = focus.parent
        
def bubble_up_to_group(focus):
    while True:
        if type(focus) == NodeGroup:
            return focus
        else:
            focus = focus.parent

# TOKEN_TYPE_STRING
# TOKEN_TYPE_INTEGER
# TOKEN_TYPE_NUMBER
# TOKEN_TYPE_LITERAL

# TOKEN_TYPE_GROUP
# TOKEN_TYPE_BINARY
# TOKEN_TYPE_UNARY_LEFT
# TOKEN_TYPE_UNARY_RIGHT

# TOKEN_TYPE_NEW_ITEM
# TOKEN_TYPE_CLOSE_GROUP

def build_token_tree(tokens):
    
    root = NodeGroup("ROOT", None)
    focus = root
    
    for token_type, value in tokens:
        
#         print('')
#         print(f"focus:{focus.value}, focus_type:{type(focus)}, value:{value}, token_type:{token_type}")
        
        
        # focus is
        # Binary Operator
        # Unary Operator
        if type(focus) in [
            NodeBinary,
            NodeUnaryLeft,
        ]:
            
            # next is 
            # Unary left
            if token_type == TOKEN_TYPE_UNARY_LEFT:
#                 print('insert lef unary')
                
                operand = find_operand(value, left_unary_operands)
                next_node = NodeUnaryLeft(operand, operand.precedence)
                
                focus.set_right(next_node)
                focus = next_node
                
            # next is
            # Group
            elif token_type == TOKEN_TYPE_GROUP:
#                 print('insert open group')
                
                group = find_operand(value, groups)
                next_node = NodeGroup(group, group.close_op)
                focus.set_right(next_node)
                focus = next_node
            
            # next is
            # TOKEN_TYPE_STRING
            # TOKEN_TYPE_INTEGER
            # TOKEN_TYPE_NUMBER
            # TOKEN_TYPE_LITERAL
            elif token_type in TOKEN_TYPE_VALUE:
#                 print('insert value')
                
                next_node = NodeValue(value, token_type)
                focus.set_right(next_node)
                focus = next_node
                
            else:
                raise Exception(f"token '{value}' not allowed here")
            
            
        # focus is
        # Value
        # Closed Group
        elif type(focus) == NodeValue or (type(focus) == NodeGroup and focus.is_complete()):
            
            # next is
            # Binary
            if token_type == TOKEN_TYPE_BINARY:
#                 print('insert binary')
                
                operand = find_operand(value, binary_operands)
                next_node = NodeBinary(operand, operand.precedence)
                    
                parent_node = bubble_up(focus.parent, next_node)
                child_node = parent_node.get_right()
                parent_node.set_right(next_node)
                next_node.set_left(child_node)
                focus = next_node
                
            
            # next is
            # Unary right
            elif token_type == TOKEN_TYPE_UNARY_RIGHT:
#                 print('insert right unary')
                
                operand = find_operand(value, right_unary_operands)
                next_node = NodeUnaryRight(operand, operand.precedence)
                    
                parent_node = focus.parent # bubble_up(focus.parent, next_node)
                child_node = parent_node.get_right()
                parent_node.set_right(next_node)
                next_node.set_left(child_node)
                # focus = next_node
            
            # next is
            # Close group
            elif token_type == TOKEN_TYPE_NEW_ITEM:
#                 print('new item')
                
                parent_node = bubble_up_to_group(focus.parent)
                parent_node.increase()
                focus = parent_node
            
            # next is
            # New item
            elif token_type == TOKEN_TYPE_CLOSE_GROUP:
#                 print('close group')
                
                parent_node = bubble_up_to_group(focus.parent)
                parent_node.close()
                focus = parent_node
            
            # next is
            else:
                raise Exception(f"token '{value}' not allowed here")
#                 print('start new item in parent group')
        
        
        # focus is
        # Open Group
        elif type(focus) == NodeGroup and not focus.is_complete(): 
            
            # next is
            # Unary left
            if token_type == TOKEN_TYPE_UNARY_LEFT:
#                 print('add unary left')
                
                operand = find_operand(value, left_unary_operands)
                next_node = NodeUnary(operand, operand.precedence)
                
                focus.add(next_node)
                focus = next_node
            
            # next is
            # Group
            elif token_type == TOKEN_TYPE_GROUP:
#                 print('add open group')
                
                group = find_operand(value, groups)
                next_node = NodeGroup(group, group.close_op)
                focus.add(next_node)
                focus = next_node
            
            # next is
            # New item
            elif token_type == TOKEN_TYPE_CLOSE_GROUP:
#                 print('close group')
                
                focus.close()
            
            # next is
            # TOKEN_TYPE_STRING
            # TOKEN_TYPE_INTEGER
            # TOKEN_TYPE_NUMBER
            # TOKEN_TYPE_LITERAL
            elif token_type in TOKEN_TYPE_VALUE:
#                 print('add value')
                
                next_node = NodeValue(value, token_type)
                focus.add(next_node)
                focus = next_node
                
            # next is 
            # TOKEN_TYPE_NEW_ITEM
            elif token_type == TOKEN_TYPE_NEW_ITEM:
#                 print('new item/dimension')
                
                focus.increase()
                
            
            # next is
            else:
                raise Exception(f"token '{value}' not allowed here")
#                 print('start new item in parent group')
            
        
    
    return root
    

## Parse

In [21]:
def build_evaluation_tree(token_tree):
    pass

## Evaluate

In [22]:
def evaluate(expression):
    return None


## Run

In [23]:
def calc(query):
        
#     commands
#     if   query == 'exit':   break
#     elif query == 'help':   help()
#     elif query == 'ref':    ref()
#     elif query == 'clear':  clear()
#     elif query == 'copy':   pyperclip.copy(ans)
#     elif query == '=':      pyperclip.copy(ans)

#     # evaluate query
#     elif query != "":

    tokens = lexing(query)
    print(tokens, end='\n\n')
    
    token_tree = build_token_tree(tokens)
    print(token_tree.dict(), end='\n\n')
    
    evaluation_tree = build_evaluation_tree(token_tree)
    print(evaluation_tree.dict(), end='\n\n')


## Test

In [24]:
calc('(1.2e-4+e) ^ -2j! -000.4e3j % fun(12cm, a, 7).abs')

[('group', '('), ('number', '1.2e-4'), ('binary operand', '+'), ('literal', 'e'), ('close group', ')'), ('binary operand', '^'), ('left unary operand', '-'), ('number', '2j'), ('right unary operand', '!'), ('binary operand', '-'), ('number', '000.4e3j'), ('binary operand', '%'), ('group', 'fun('), ('number', '12cm'), ('new item', ','), ('literal', 'a'), ('new item', ','), ('number', '7'), ('close group', ')'), ('binary operand', '.'), ('literal', 'abs')]

{('ROOT', (1,)): [{Operand(\-, 3.0): {'left': {Operand(\^, 5.0): {'left': {(Group(\(, )), (1,)): [{Operand(\+, 3.0): {'left': '1.2e-4', 'right': 'e'}}]}, 'right': {Operand(\-, 7.0): {'child': {Operand(\!, 7.0): {'child': '2j'}}}}}}, 'right': {Operand(\%, 4.0): {'left': '000.4e3j', 'right': {Operand(\., 8.0): {'left': {(Group([a-zA-Z][a-zA-Z0-9_]*\(, )), (3,)): ['12cm', 'a', '7']}, 'right': 'abs'}}}}}}]}



AttributeError: 'NoneType' object has no attribute 'dict'